<a href="https://colab.research.google.com/github/bachvu98/Policy-NLP/blob/master/Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Basic imports
from os import listdir
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import *
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

import json
import time

In [11]:
url = "/content/drive/My Drive/OPP-115/OPP-115/annotations/"
files = [f for f in listdir(url) if f.endswith('.csv')]

In [31]:
names = """annotation_id 
        batch_id 
        annotator_id 
        policy_id 
        segment_id 
        category_name 
        attributes_value_pairs 
        date 
        policy_url""".split()

types = {'annotation_id': np.int,
        'batch_id': str,
        'annotator_id': np.int,
        'policy_id': np.int,
        'segment_id': np.int,
        'category_name': str}

annotations = pd.concat(pd.read_csv(url + f,header=None,names=names,
             na_values={'date': 'Not specified'}, parse_dates=[7],index_col=0) for f in files)

In [36]:
attr_values = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
template = dict.fromkeys('startIndexInSegment endIndexInSegment selectedText value'.split())

In [ ]:
attr_values1 = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
t0 = time.time()
print('Starting at ' + str(t0))
for i in range(20000,annotations.shape[0]):    #Note manual adjustment of batch size here...
    
    attr_val = annotations['attributes_value_pairs'].iloc[i]
    ann_id = annotations.index.values[i]
    
    obj = json.loads(attr_val)
    keys = list(obj.keys())
    
    for k in keys:
        obj2 = dict(template, **obj[k]) #Ensures at a minimum we get the empty template data.
        df = pd.DataFrame({'annotation_id': [ann_id],
                           'start_idx': [obj2['startIndexInSegment']],
                           'end_idx': [obj2['endIndexInSegment']],
                           'attribute': [k],
                           'text': [obj2['selectedText']],
                           'value': [obj2['value']]})
        attr_values1 = attr_values1.append(df,ignore_index=True)
t1 = time.time()
print('Finished at ' + str(t1))
print('Total elapsed time ' + str(t1-t0))
attr_values = attr_values.append(attr_values1,ignore_index=True)
attr_values.info()

In [65]:
attr_values

,annotation_id,start_idx,end_idx,attribute,text,value
0,3844,-1,-1,Collection Mode,null,not-selected
1,3844,223,238,Choice Scope,use and sharing,Use
2,3844,95,105,Action First-Party,we collect,Collect on website
3,3844,74,123,Personal Information Type,any information that we collect from or about you,Unspecified
4,3844,191,253,Choice Type,", which provides choices in the use and sharin...",Unspecified
...,...,...,...,...,...,...
7008,3847,127,157,Notification Type,we will update it on this page,General notice in privacy policy
7009,3847,193,249,Change Type,If our information practices change in a signi...,Privacy relevant change
7010,3847,440,642,User Choice,If you disagree with any changes to this priva...,Opt-out
7011,3847,251,290,Notification Type,we will provide a more prominent notice,Unspecified
